In [20]:
%matplotlib inline

import json
Settings = json.load(open('../settings.txt'))
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
from cselect import color as cs
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../samples')
from mvpose.data import epfl_campus
from time import time

root = Settings['data_root']
root = join(root, 'pak')
tmp = Settings['tmp']

import mvpose.data.kth_football2 as kth
from mvpose import pose
from mvpose.settings import get_settings
from paf_loader import Loader
from mvpose.evaluation import pcp

# =====================================
params = get_settings(scale_to_mm=1000)
loader = Loader()
# =====================================

# def execute(frame):
#     global params, loader
#     _start = time()
#     Im, Y, Calib = epfl_campus.get(root, frame)
#     heatmaps, pafs = loader.load_confidence_map_and_paf(
#         'campus', Im, frame, dir=Settings['tmp'])
#     _end = time()
#     print('frame ' + str(frame) + ', elapsed:', _end - _start)

    
def generate_pcp_score(frame):
    global Settings, params
    alpha = 0.5
    
    Im, Y, Calib = epfl_campus.get(root, frame)
    heatmaps, pafs = loader.load_confidence_map_and_paf(
        'campus', Im, frame, dir=Settings['tmp'])
    detections = pose.estimate(Calib, heatmaps, pafs, 
                               settings=params, debug=False)
    
    Humans = kth.transform3d_from_mscoco(detections)
    
    L_Arms = []
    U_Arms = []
    L_Legs = []
    U_Legs = []
    GTIDs = []
  
    for gtid, gt in enumerate(Y):
        if gt is None:
            continue
        
        larms = 0
        uarms = 0
        llegs = 0
        ulegs = 0
        avg = 0
        for d in Humans:
            r = pcp.evaluate(gt, d, alpha)
            larms_ = r.lower_arms
            uarms_ = r.upper_arms
            ulegs_ = r.upper_legs
            llegs_ = r.lower_legs
            avg_ = (larms_ + uarms_ + ulegs_ + llegs_) / 4
            if avg_ > avg:
                avg = avg_
                larms = larms_
                uarms = uarms_
                llegs = llegs_
                ulegs = ulegs_
        
        L_Arms.append(larms)
        U_Arms.append(uarms)
        L_Legs.append(llegs)
        U_Legs.append(ulegs)
        GTIDs.append(gtid)
    
    del heatmaps
    del pafs
    del detections
    del Im
    del Y
    del Calib
    
    return L_Arms, U_Arms, L_Legs, U_Legs, GTIDs

# =====================================

valid_frames = list(range(350, 470)) + list(range(650, 750))
PER_GTID = {}
for frame in valid_frames:
    _start = time()
    L_Arms, U_Arms, L_Legs, U_Legs, GTIDs =\
        generate_pcp_score(frame)
    _end = time()
    print('frame ' + str(frame) + ', elapsed:', _end - _start)
    if len(L_Arms) > 0:
        for gtid, larms, uarms, llegs, ulegs in zip(
            GTIDs, L_Arms, U_Arms, L_Legs, U_Legs
        ):
            if not gtid in PER_GTID:
                PER_GTID[gtid] = {
                    'larms': [],
                    'uarms': [],
                    'llegs': [],
                    'ulegs': [],
                    'frame': []
                }
            PER_GTID[gtid]['larms'].append(larms)
            PER_GTID[gtid]['uarms'].append(uarms)
            PER_GTID[gtid]['llegs'].append(llegs)
            PER_GTID[gtid]['ulegs'].append(ulegs)
            PER_GTID[gtid]['frame'].append(frame)

frame 350, elapsed: 0.3064119815826416
frame 351, elapsed: 0.31023097038269043
frame 352, elapsed: 0.2912774085998535
frame 353, elapsed: 0.3038456439971924
frame 354, elapsed: 0.30898237228393555
frame 355, elapsed: 1.0714104175567627
frame 356, elapsed: 1.0873465538024902
frame 357, elapsed: 1.0835416316986084
frame 358, elapsed: 1.0632905960083008
frame 359, elapsed: 1.1812443733215332
frame 360, elapsed: 1.0778484344482422
frame 361, elapsed: 1.0612006187438965
frame 362, elapsed: 1.0852184295654297
frame 363, elapsed: 1.0702919960021973
frame 364, elapsed: 1.0567378997802734
frame 365, elapsed: 1.1942110061645508
frame 366, elapsed: 1.0722935199737549
frame 367, elapsed: 1.0589854717254639
frame 368, elapsed: 1.069232702255249
frame 369, elapsed: 1.0838632583618164
frame 370, elapsed: 1.1695928573608398
frame 371, elapsed: 1.0683391094207764
frame 372, elapsed: 1.068744421005249
frame 373, elapsed: 1.0396718978881836
frame 374, elapsed: 1.057814359664917
frame 375, elapsed: 1.1518

frame 742, elapsed: 1.454615831375122
frame 743, elapsed: 1.3979406356811523
frame 744, elapsed: 1.4204702377319336
frame 745, elapsed: 1.3669991493225098
frame 746, elapsed: 2.40020489692688
frame 747, elapsed: 1.5167396068572998
frame 748, elapsed: 2.439085006713867
frame 749, elapsed: 1.3320622444152832


In [24]:
total_avg = []
for key, values in PER_GTID.items():
    print('actor ', key)
    print('\tuarms:', np.mean(values['uarms']))
    print('\tlarms:', np.mean(values['larms']))
    print('\tulegs:', np.mean(values['ulegs']))
    print('\tllegs:', np.mean(values['llegs']))
    avg = np.mean([
        np.mean(values['uarms']),
        np.mean(values['larms']),
        np.mean(values['ulegs']),
        np.mean(values['llegs'])
    ])
    total_avg.append(avg)
    print('\tavg:  ', avg)
print('\navg*:  ', np.mean(total_avg))

    

actor  1
	uarms: 0.9813829787234043
	larms: 0.601063829787234
	ulegs: 1.0
	llegs: 0.9973404255319149
	avg:   0.8949468085106383
actor  2
	uarms: 0.9007352941176471
	larms: 0.6544117647058824
	ulegs: 0.9742647058823529
	llegs: 0.9742647058823529
	avg:   0.8759191176470588
actor  0
	uarms: 0.6979166666666666
	larms: 0.5104166666666666
	ulegs: 0.9583333333333334
	llegs: 0.9583333333333334
	avg:   0.78125

avg*:   0.8507053087192323
